#### Импорт необходимых модулей

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from bs4 import BeautifulSoup
import sqlite3

#### Подключение к базе данных

In [2]:
conn = sqlite3.connect('reviews.db')
cur = conn.cursor()

#### Получение отзывов с сайта IMDB

Поскольку отзывов на одной страницы меньше, чем нужно нам, воспользуемся Selenium для имитации нажатия кнопки "Load more" внизу страницы. В зависимости от того, парсим мы наши "обучающие" или "тестовые" отзывы, определим число итераций: по 20 страниц положительных и отрицательных отзывов на Интерстеллар, на котором будем обучаться, и по 5 страниц положительных и отрицательных отзывов для Марсианина, на котором будем проверять модель.

In [3]:
def getReviews(url, polarity, mode, pages):
    driver = webdriver.Edge(r'C:\Users\Igor\Desktop\py_hw\test\msedgedriver.exe')
    wait = WebDriverWait(driver, 10)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')

    i = 0
    while i < pages:
        i += 1
        try:
            driver.find_element_by_css_selector('button#load-more-trigger').click()
            wait.until(ec.invisibility_of_element_located((By.CSS_SELECTOR, '.ipl-load-more__load-indicator')))
            soup = BeautifulSoup(driver.page_source, 'lxml')
        except:
            pass

    for elem in soup.find_all(class_='imdb-user-review'):
        try:
            review = elem.find(class_='text show-more__control').get_text(strip=True)
            if mode == 'reviews':
                cur.execute('INSERT INTO reviews VALUES (?, ?)', (review, polarity))
                conn.commit()
            if mode == 'test':
                cur.execute('INSERT INTO test VALUES (?, ?)', (review, polarity))
                conn.commit()
        except:
            pass

    driver.quit()

Запишем полученные отзывы в базу данных, чтобы не парсить каждый раз всё заново и не нарваться на блокировку: в таблице reviews будем хранить обучающие отзывы, а в таблице test - отзывы, полярность которых будем пытаться предсказать.

In [4]:
getReviews('https://www.imdb.com/title/tt0816692/reviews?spoiler=hide&sort=submissionDate&dir=desc&ratingFilter=10',
           'pos', 'reviews', 20)
getReviews('https://www.imdb.com/title/tt0816692/reviews?spoiler=hide&sort=submissionDate&dir=desc&ratingFilter=1',
           'neg', 'reviews', 20)

getReviews('https://www.imdb.com/title/tt3659388/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=10',
           'pos', 'test', 5)
getReviews('https://www.imdb.com/title/tt3659388/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=1',
           'neg', 'test', 5)